# Maximal Static Expansion for efficient parallelization on GPU

## Context

### Polly
My GSoC project is part of Polly. Polly is a loop and data-locality optimizer for LLVM. The optimisations are made using a mathematical model called polyhedral model. It models the memory access of the program. After modeling, transformations (tilling, loop fusion, loop unrolling ...) can be applied on the model to improve data-locality and/or parallelization. The aim of my project was to implement a transformation called Maximal Static Expansion (MSE).

### Maximal static expansion
Data-dependences in a program can lead to a very bad automatic parallelization. Modern compilers use techniques to reduce the number of such dependences. One of them is Maximal Static Expansion. The MSE is a transformation which expand the memory access to and from Array or Scalar. The goal is to disambiguate memory accesses by assigning different memory locations to non-conflicting writes. This method is described in a paper written by Denis Barthou, Albert Cohen and Jean-Francois Collard.[^f1] 
Let take a example (from the article) to understand the principle :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        tmp = tmp + i + j;
    }
    A[i] = tmp;
}

The data-dependences induced by tmp make the two loops unparallelizable : the iteration j of the inner-loop needs value from the previous iteration and it is impossible to parrallelize the i-loop because tmp is used in all iterations.

If we expand the accesses to tmp according to the outermost loop, we can then parallelize the i-loop.

In [ ]:
int tmp_exp[N];
for (int i = 0; i < N; i++) {
    tmp_exp[i] = i;
    for (int j = 0; j < N; j++) {
        tmp_exp[i] = tmp_exp[i] + i + j;
    }
    A[i] = tmp_exp[i];
}

The accesses to tmp are now made to/from a different location for each iteration of the i-loop. It is then possible to execute the different iteration on different computation units (GPU, CPU ...).

### Static single assignement
Due to lack of time, I was not able to implement **maximal** static expansion but only fully-indexed expansion. The principle of fully-index expansion is that each write goes to a different memory location. 
Let see the idea on an example :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B[j] = tmp + 3;
    }
    A[i] = B[i];
}

For the sake of simplicity, only the arrays will be expanded in this example. The fully expanded version is :

In [ ]:
int tmp;
for (int i = 0; i < N; i++) {
    tmp = i;
    for (int j = 0; j < N; j++) {
        B_exp[i][j] = tmp + 3;
    }
    A_exp[i] = B_exp[i][i];
}

The details of fully-indexed expansion will be discussed in the following sections.


## My work
My project is part of Polly. I am a french student but during the GSoC I was a student at the university of Passau, Germany. The LooPo team welcome me and more especially Andreas Simbürger, one of my GSoC mentor and my master thesis supervisor. My other GSoC mentor is Michael Kruse, one of the main contributor to Polly, actually working in France.
I'd like to thank all the people that help and guide me and more especially Andreas and Michael. 

### JSON bug fix
As first step in open source software development and to get familiar with Polly/LLVM development process, I fixed a open bug in Polly. Polly can import data from a JSON file (in case of Polly called jscop file). It can import new array, new memory access, new schedule or new context. In the previous implementation of JSONImporter, Polly did not check if the data in the jscop file were plausible and consistent before import. This can lead to failure in the remaining part of the Polly pipeline. My work was to implement plausibility and consistency checks. Details, diff and discussions can be found here : https://reviews.llvm.org/D32739. This patch has been merged into the actual version of Polly.

### Allocate array on heap
During transformation, Polly can create new arrays because it helps optimizing the program. Before GSoC, it was only possible to allocate array on the stack. It is sufficient for small arrays, but while doing expansion, we possibly handle very large arrays. For example, if we want to expand fully this simple code :


In [ ]:
for (int i = 0; i < N; i++)
  for (int j = 0; j < N; j++)
    for (int k = 0; k < N; k++)
      for (int l = 0; l < N; l++)
        A[l] = 3;

The expansion would lead to the following code :

In [ ]:
for (int i = 0; i < N; i++)
  for (int j = 0; j < N; j++)
    for (int k = 0; k < N; k++)
      for (int l = 0; l < N; l++)
        A_exp[i][j][k][l] = 3;

Depending on the value of N, A_exp can have a huge number of elements. If N = 100, we have $100*100*100*100 = 100000000 = 10^8$ elements ! Thus, the possibility to allocate array on heap was needed. 

The array allocation is implemented in the IslNodeBuilder. Here is the part of the code that do the allocation :

In [ ]:
void IslNodeBuilder::allocateNewArrays(BBPair StartExitBlocks) {
  for (auto &SAI : S.arrays()) {
    if (SAI->getBasePtr())
      continue;

    assert(SAI->getNumberOfDimensions() > 0 && SAI->getDimensionSize(0) &&
           "The size of the outermost dimension is used to declare newly "
           "created arrays that require memory allocation.");

    Type *NewArrayType = nullptr;

    // Get the size of the array = size(dim_1)*...*size(dim_n)
    uint64_t ArraySizeInt = 1;
    for (int i = SAI->getNumberOfDimensions() - 1; i >= 0; i--) {
      auto *DimSize = SAI->getDimensionSize(i);
      unsigned UnsignedDimSize = static_cast<const SCEVConstant *>(DimSize)
                                     ->getAPInt()
                                     .getLimitedValue();

      if (!NewArrayType)
        NewArrayType = SAI->getElementType();

      NewArrayType = ArrayType::get(NewArrayType, UnsignedDimSize);
      ArraySizeInt *= UnsignedDimSize;
    }

    if (SAI->isOnHeap()) {
      LLVMContext &Ctx = NewArrayType->getContext();

      // Get the IntPtrTy from the Datalayout
      auto IntPtrTy = DL.getIntPtrType(Ctx);

      // Get the size of the element type in bits
      unsigned Size = SAI->getElemSizeInBytes();

      // Insert the malloc call at polly.start
      auto InstIt = std::get<0>(StartExitBlocks)->getTerminator();
      auto *CreatedArray = CallInst::CreateMalloc(
          &*InstIt, IntPtrTy, SAI->getElementType(),
          ConstantInt::get(Type::getInt64Ty(Ctx), Size),
          ConstantInt::get(Type::getInt64Ty(Ctx), ArraySizeInt), nullptr,
          SAI->getName());

      SAI->setBasePtr(CreatedArray);

      // Insert the free call at polly.exiting
      CallInst::CreateFree(CreatedArray,
                           std::get<1>(StartExitBlocks)->getTerminator());

    } else {
      auto InstIt = Builder.GetInsertBlock()
                        ->getParent()
                        ->getEntryBlock()
                        .getTerminator();

      auto *CreatedArray = new AllocaInst(NewArrayType, DL.getAllocaAddrSpace(),
                                          SAI->getName(), &*InstIt);
      CreatedArray->setAlignment(PollyTargetFirstLevelCacheLineSize);
      SAI->setBasePtr(CreatedArray);
    }
  }
}

My work in this method is the size computation and the heap allocation part. The remaining code was already in place.

Let explain step by step the principle of the heap allocation.

First of all, to allocate array, we need to have the size of the memory chunk we want to allocate. To do that, we simply iterate over the dimension of the ScopArrayInfo and multiply the size of each dimensions. This is done by this code :

In [ ]:
    // Get the size of the array = size(dim_1)*...*size(dim_n)
    uint64_t ArraySizeInt = 1;
    for (int i = SAI->getNumberOfDimensions() - 1; i >= 0; i--) {
      auto *DimSize = SAI->getDimensionSize(i);
      unsigned UnsignedDimSize = static_cast<const SCEVConstant *>(DimSize)
                                     ->getAPInt()
                                     .getLimitedValue();

Details, diff and discussions can be found here : https://reviews.llvm.org/D33688. This patch has been merged into the actual version of Polly.

### Array Fully indexed exp
https://reviews.llvm.org/D34982
bug : https://reviews.llvm.org/D36791
clear deps : https://reviews.llvm.org/D36926

### Scalar Fully indexed exp
Details, diff and discussions can be found here : https://reviews.llvm.org/D36647. This patch has been merged into the actual version of Polly.


## Evaluation

## Remaining work

### MAXIMAL expansion
### Select which SAI to expand

[^f1]: Denis Barthou, Albert Cohen, and Jean-François Collard. 2000. Maximal Static Expansion. Int. J. Parallel Program. 28, 3 (June 2000), 213-243. DOI=http://dx.doi.org/10.1023/A:1007500431910 
